# PO : Optimization of Complex Decisions - 
# Project : Optimization of the maintenance planning of a fleet of offshore wind turbine

### Sets

 $H$ = horizons de temps

 $S$ = Set du personnel de staffing

 $Q$ = Set de compétences 

$P$ = Set de projets

### Parametres

A. Costs 

-  $C = (c_{s,q})_{(s,q)\in S \times Q}$     = cost of a preventive maintenance
-  $c_{cm}$     = cost of a corrective maintenance
-  $c_{op}$     = opportunity cost
-  $c_{transport}$     = cost of transport from a wind farm to another

B. Wind farm characteristics

- N = number of wind farms ($=len(I)$)
- $n_i$     = number of turbines within the wind farm i ($=len(J_{i})$ , $\forall i \in I$)
- H     = wind turbines lifetime
- h     =defining a time window within which the operating crew can proceed to a preventive maintenance

C. Maintenance 

- R     = number of maintenance crews ($=len(O)$)
- k     = maximum number of maintenance operations within a farm during a single time step 
- tw     = time window for the maintenance planning ($=len(T) - 1$)

### Variables

A. State of the turbines

Each of the following parameters indexed i,j represent a characteristic of the state at time t of the turbine j within the wind farm i :

-  $h_{t,i,j}$     = operating time 
-  $pm_{t,i,j}$     = True if there is a preventive maintenance occuring
-  $cm_{t,i,j}$     = True if there is a corrective maintenance occuring
-  $f_{t,i,j}$     = True if a failure occured (i.e $h_{i,j} ≥ H$)

B. Position of the maintenance crews

At time t, we consider the following booleans :

- $w_{t,o,i}$ : True if the operatoring crew o is currently present at the wind farm i
- $tr_{t,o,i}$ : True if the operatoring crew o is currently transporting from any wind farm to the wind farm i

### Objective

Let's define V as the ensemble of variables previously mentionned. 

Then we try to minimize the total cost of the fleet of offshore wind turbines: <br />

$\min_{variables \in V} Z = \sum_{t = 0}^{tw} \sum_{i \in I} (\sum_{j = 1}^{n_{i}} [c_{pm} pm_{t,i,j} + c_{cm} cm_{t,i,j}+ c_{op} f_{t,i,j}] + \sum_{o = 1}^{R} c_{transport} tr_{t,o,i}) $

### Constraints

We fix $M = 10^{3}$ for applying the Big-M method. 

Type of the parameters : $\forall (t,(i,j),o) \in T \times IJ \times O$, 

- $h_{t,i,j}\in\mathbb{N}$
- $(pm_{t,i,j},cm_{t,i,j},f_{t,i,j},w_{t,o,i},tr_{t,o,i}) \in \{0,1\}^{5}$

<u><strong>A. Constraints on turbine states</strong></u>

Not more than k operations per operating team on a single wind farm during a single time step : <br />

 $\sum_{j \in J_{i}} (pm_{t,i,j} + cm_{t,i,j}) ≤ k \sum_{o \in O} w_{t,o,i}$ ,  $\forall (t,i) \in T \times I$

Then $\forall (t,(i,j)) \in T \times IJ$, 

Only one state at a time for each turbine : <br />

$pm_{t,i,j} + cm_{t,i,j}+ f_{t,i,j} ≤ 1$

 A preventive maintenance can be done only during a precise operating window ($H - h ≤ h_{t,i,j} ≤ H - 1$) :

 - $pm_{t,i,j} ≤ \frac{h_{t,i,j}}{H-h}$
 - $pm_{t,i,j} ≤ 1 + M^{-1}(H-1-h_{t,i,j})$

When $h_{t,i,j} ≥ H$ then the system goes either in failure state or in CM state :

$f_{t,i,j} + cm_{t,i,j} ≥ M^{-1}(h_{t,i,j}-H+1)$

Otherwise the system can not be in these states : 

$f_{t,i,j} ≤ 1 + M^{-1}(h_{t,i,j}-H)$

<u><strong>B. Relationship between timestep t and t-1 :</strong></u>

$\forall (t,(i,j)) \in T^* \times IJ$,

If a maintenance occured during the previous timestep the operation time comes back to 0 :

- $h_{t,i,j} ≤ M(1-pm_{t-1,i,j}-cm_{t-1,i,j})$
- $h_{t,i,j} ≥ M(1-pm_{t-1,i,j}-cm_{t-1,i,j})$

Otherwise we just increment by 1 each time step : 

- $h_{t,i,j} - h_{t-1,i,j} ≥ 1 - M(pm_{t-1,i,j}+cm_{t-1,i,j})$
- $h_{t,i,j} - h_{t-1,i,j} ≤ 1 + M(pm_{t-1,i,j}+cm_{t-1,i,j})$

An operating team can be at wind farm i only if it was there in the previous time step or was traveling there : 

- $w_{t,o,i} ≤ w_{t-1,o,i} + tr_{t-1,o,i}$ , $\forall (t,o,i) \in T^* \times O \times I$

<u><strong>C. Maintenance constraints :</strong></u>

An operating team is at one place at a time :

$\sum_{i \in I} (w_{t,o,i} + tr_{t,o,i}) ≤ 1$ , $\forall (t,o) \in T \times O$